In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io, filters
import numpy as np

from tqdm.notebook import tqdm

import cv2
import copy
import time

from mpire import WorkerPool
from StereoFractAnalyzer import StereoFractAnalyzer

from PIL import Image
from matplotlib import cm

# from combra.stats.preprocess import stats_preprocess

from combra import stats as cstats
from combra import approx as capprox

In [ ]:
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (3-10-2024_18-11--6-46)/IMG_1259.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (3-10-2024_18-11--6-46)/IMG_2266.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (24-09-2024_12-27--1-0)/IMG_6186.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (24-09-2024_12-27--1-0)/IMG_5589.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (12-09-2024_12-26--1-3)/IMG_4097.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (12-09-2024_12-26--1-3)/IMG_3603.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (1-10-2024_14-6--2-40)/IMG_0585.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (1-10-2024_14-6--2-40)/IMG_0268.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (09-09-2024_12-10--0-44)/IMG_9009.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (09-09-2024_12-10--0-44)/IMG_8955.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (02-09-2024_13-33--2-7)/IMG_8747.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (02-09-2024_13-33--2-7)/IMG_6932.JPG'

# img = io.imread(image_path)
# img = img[:,900:-900]

# original_img = original_img[500:-300,1200:-1500]
# original_img=original_img[:2000,:2000]

# plt.imshow(img)
# plt.show()

# Image preprocess

In [ ]:
image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (09-09-2024_12-10--0-44)/IMG_8955.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (02-09-2024_13-33--2-7)/IMG_8747.JPG'


img = io.imread(image_path)
img = img[500:-300,1200:-1500]

img=img[:2000,:2000]
img[:250,:250]=np.full((250,250,3),20)

plt.imshow(img)
plt.show()

In [ ]:
images_paths=['images/0001.JPG',
              'images/0500.JPG',
              'images/1000.JPG',
              'images/1500.JPG',
              'images/2000.JPG',
              'images/2500.JPG',
              'images/3000.JPG',
              'images/3500.JPG',
              'images/4000.JPG',
            ]
img = io.imread(images_paths[2])
img = img[400:-400,1300:-1300]

print(img.shape)


thresh = filters.threshold_otsu(img)
binary = img > thresh  # Apply threshold
binary = binary*255


plt.imshow(binary)
plt.show()

In [ ]:

def find_contours(binary_image):
    """Find contours using OpenCV"""
    contours, _ = cv2.findContours(binary_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_contours(original_img, contours):
    """Draw contours on the original image"""
    contour_img = cv2.cvtColor(original_img, cv2.COLOR_GRAY2BGR)
    for contour in contours:
        color=(random.randint(0,255),random.randint(0,255),random.randint(0,255))
        cv2.drawContours(contour_img, contour, -1, color, 2)
    return contour_img

def add_text_upper_right(img, text, font_scale=1.0, thickness=2, color=(255, 255, 255), 
                         bg_color=(0, 0, 0), padding=10):
    """Add text to the upper right corner of the image"""
    # Get image dimensions
    h, w = img.shape[:2]
    
    # Set font
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # Get text size
    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
    
    # Calculate position (upper right corner)
    x = w - text_width - padding
    y = text_height + padding
    
    # Optional: Draw background rectangle for better visibility
    if bg_color is not None:
        cv2.rectangle(img, 
                     (x - padding//2, y - text_height - padding//2), 
                     (x + text_width + padding//2, y + baseline + padding//2), 
                     bg_color, -1)
    
    # Draw text
    cv2.putText(img, text, (x, y), font, font_scale, color, thickness, cv2.LINE_AA)
    
    return img

def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = img[400:-400,1300:-1300]
    _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    cnts = find_contours(binary)
    return binary, cnts

def plot_contours(img, contours, images_path):

    contour_img = draw_contours(img, contours)

    # 3. Add text to upper right corner
    text = f"{images_path.split('/')[-1]}"
    contour_img = add_text_upper_right(contour_img, text, font_scale=7.5, thickness=2, 
                                    color=(255, 255, 255), bg_color=(0, 0, 0), padding=10)

    # fig, axes = plt.subplots(1,2,figsize=(20,10))

    # axes[0].imshow(img, cmap='gray')
    # axes[1].imshow(contour_img)
    # axes[1].set_title(f'number of contours {len(contours)}')
    # cv2.imwrite(f'images/contours_{images_path.split("/")[-1]}', contour_img)

# Main processing
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (09-09-2024_12-10--0-44)/IMG_8955.JPG'
images_paths=[
            #   'data/images/0005.JPG',
              'data/images/0500.JPG',
              'data/images/1000.JPG',
              'data/images/1500.JPG',
              'data/images/2000.JPG',
              'data/images/2500.JPG',
              'data/images/3000.JPG',
              'data/images/3500.JPG',
              'data/images/4000.JPG',
              'data/images/4321.JPG',
              ]

images_names=[os.path.basename(image_path) for image_path in images_paths]

for image_path in images_paths:
    binary, cnts = preprocess_image(image_path)
    plot_contours(binary, cnts, image_path)


# Contour length

In [ ]:
%%time

types=['len_nodes', 'len_pixels', 'area']
type=types[1]

# step=1

for step in tqdm([1,2,3,4,5]):
# for step in tqdm([1,2]):
    legend=[]

    fig, axes= plt.subplots(2,4,figsize=(40,14))
    
    # Define 10 distinct colors
    colors = plt.cm.tab10(np.linspace(0, 1, 10))
    
    n_list=[]
    p_list=[]
    lambda_list=[]
    mu_list=[]
    sigma_list=[]
    a_list=[]

    for idx, image_path in enumerate(images_paths):
        binary, cnts = preprocess_image(image_path)
        plot_contours(binary, cnts, image_path)

        len_list=[]

        for contour in cnts:
            if type=='len_nodes':
                len_list.append(len(contour))
            elif type=='len_pixels':
                len_list.append(cv2.arcLength(contour, closed=True))
            elif type=='area':
                len_list.append(cv2.contourArea(contour))

        x_orig, y_orig = cstats.stats_preprocess(len_list, step)
        
        # Calculate mode (x value with maximum y)
        mode_idx = np.argmax(y_orig)
        mode = x_orig[mode_idx]
        
        # Get color for this image (cycle through colors if more than 10 images)
        color = colors[idx % len(colors)]

        # Fit binomial distribution
        (x_fit, y_fit), n, p, amp = capprox.binomial_approx(x_orig, y_orig, n=25,p=0.2, x_lim=[0,25], N=100)
        axes[0,0].plot(x_fit, y_fit, ':', linewidth=2, color=color)

        # Fit Poisson distribution
        (x_fit, y_fit), lam, amp_poisson = capprox.poisson_approx(x_orig, y_orig, x_lim=[-5,25], N=100)
        axes[0,1].plot(x_fit, y_fit, ':', linewidth=2, color=color)

        # Fit Gaussian distribution
        (x_fit, y_fit), mu, sigma, amp_gauss = capprox.gauss_approx(x_orig, y_orig, mu=3, sigma=3, amp=1, x_lim=[0,25], N=100)
        axes[0,2].plot(x_fit, y_fit, '-.', linewidth=2, color=color)

        # Fit Exponential distribution
        (x_fit, y_fit), a, amp_exp = capprox.exponential_approx(x_orig, y_orig, a=5, amp=1, x_lim=[0,25], N=100)
        axes[0,3].plot(x_fit, y_fit, '--', linewidth=2, color=color)

        axes[0, 0].set_title(f'binomial fit, step={step}',fontsize=15)
        axes[0,1].set_title(f'poisson fit, step={step}',fontsize=15)
        axes[0,2].set_title(f'gaussian fit, step={step}',fontsize=15)
        axes[0,3].set_title(f'exponential fit, step={step}',fontsize=15)
        
        n_list.append(n)
        p_list.append(p)
        lambda_list.append(lam)
        mu_list.append(mu)
        sigma_list.append(sigma)
        a_list.append(a)

        # Plot original data with the same color (include mode in legend)
        axes[0,0].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, n={n:.0f}, p={p:.3f}')
        axes[0,1].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, λ={lam:.2f}' )
        axes[0,2].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, μ={mu:.2f}, σ={sigma:.2f}' )
        axes[0,3].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, a={a:.2f}' )
    
    
    axes[1,0].set_title(f'binomial fit, step={step}',fontsize=15)
    axes[1,1].set_title(f'poisson fit, step={step}',fontsize=15)
    axes[1,2].set_title(f'gaussian fit, step={step}',fontsize=15)
    axes[1,3].set_title(f'exponential fit, step={step}',fontsize=15)

    # Use dual y-axis for n and p (n on left, p on right)
    ax1 = axes[1,0]  # Left axis for n
    ax1.plot(range(len(images_paths)), n_list, '-o', label='n', color='blue')
    ax1.set_ylabel('n', color='blue', fontsize=15)
    ax1.tick_params(axis='y', labelcolor='blue', labelsize=15)
    # axes[1,0].set_yscale('log')
    
    # Create second y-axis on the right for p
    ax2 = ax1.twinx()  # Right axis for p
    ax2.plot(range(len(images_paths)), p_list, '-o', label='p', color='red')
    ax2.set_ylabel('p', color='red', fontsize=15)
    ax2.tick_params(axis='y', labelcolor='red', labelsize=15)
    axes[1,0].twinx().set_yscale('log')
    
    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=15)
    
    axes[1,1].plot(range(len(images_paths)), lambda_list, '-o', label='λ')
    axes[1,2].plot(range(len(images_paths)), mu_list, '-o', label='mu')
    axes[1,2].plot(range(len(images_paths)), sigma_list, '-o', label='sigma')
    axes[1,3].plot(range(len(images_paths)), a_list, '-o', label='a')

    for ax in [axes[1,0], axes[1,1], axes[1,2], axes[1,3]]:
        ax.set_xticks(range(len(images_paths)))
        ax.set_xticklabels(images_names, rotation=45, fontsize=15, ha='right')

    axes[0,0].legend( loc='upper right',fontsize=15)
    axes[0,1].legend( loc='upper right',fontsize=15)
    axes[0,2].legend( loc='upper right',fontsize=15)
    axes[0,3].legend( loc='upper right',fontsize=15)
    axes[1,1].legend( loc='upper right',fontsize=15)
    axes[1,2].legend( loc='upper right',fontsize=15)
    axes[1,3].legend( loc='upper right',fontsize=15)

    axes[0,0].set_xlim(0,25)
    axes[0,1].set_xlim(0,25)
    axes[0,2].set_xlim(0,25)
    axes[0,3].set_xlim(0,25)

    if type=='len_nodes':
        axes[0,0].set_xlabel('len of contour, nodes',fontsize=15)
        axes[0,1].set_xlabel('len of contour, nodes',fontsize=15)
        axes[0,2].set_xlabel('len of contour, nodes',fontsize=15)
        axes[0,3].set_xlabel('len of contour, nodes',fontsize=15)
    elif type=='len_pixels':
        axes[0,0].set_xlabel('len of contour, pixels',fontsize=15)
        axes[0,1].set_xlabel('len of contour, pixels',fontsize=15)
        axes[0,2].set_xlabel('len of contour, pixels',fontsize=15)
        axes[0,3].set_xlabel('len of contour, pixels',fontsize=15)
    elif type=='area':
        axes[0,0].set_xlabel('area of contour, pixels',fontsize=15)
        axes[0,1].set_xlabel('area of contour, pixels',fontsize=15)
        axes[0,2].set_xlabel('area of contour, pixels',fontsize=15)
        axes[0,3].set_xlabel('area of contour, pixels',fontsize=15)

    axes[0,0].set_ylabel('p(x)',fontsize=15)
    axes[0,1].set_ylabel('p(x)',fontsize=15)
    axes[0,2].set_ylabel('p(x)',fontsize=15)
    axes[0,3].set_ylabel('p(x)',fontsize=15)

    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    if type=='len_nodes':
        plt.savefig(f'countour_lens_nodes_step={step}.jpg',bbox_inches='tight')
    elif type=='len_pixels':
        plt.savefig(f'countour_lens_pixels_step={step}.jpg',bbox_inches='tight')
    elif type=='area':
        plt.savefig(f'countour_areas_step={step}.jpg',bbox_inches='tight')

    plt.show()

# Area of contours

In [ ]:
%%time

legend=[]

for step in [1,2,3,4,5]:

    plt.figure(figsize=(10,7))


    for image_path in tqdm(images_paths):
        binary, cnts = preprocess_image(image_path)
        plot_contours(binary, cnts, image_path)

        area_list=[]

        for contour in cnts:
            area_list.append(cv2.contourArea(contour))

        x, y = combra_stats.stats_preprocess(area_list, step)
        plt.plot(x, y, '-o')

        legend.append(f'{len(cnts)} contours {image_path.split("/")[-1]}')


    plt.legend(legend, loc='upper right',fontsize=15)
    plt.xlim(0,40)
    # plt.xscale('log')
    # plt.yscale('log')
    plt.xlabel('area of contour, pixels',fontsize=15)
    plt.ylabel('p(x)',fontsize=15)
    plt.title(f'step={step}',fontsize=15)

    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    # plt.savefig(f'countour_areas_step={step}.jpg',bbox_inches='tight')

    plt.show()

# Fractal dimention

In [ ]:
def process_contour(contour_idx):

    contour = contours[contour_idx]

    eps=3
    x_min,y_min=np.min(contour,axis=0)[0]

    cnt_new = copy.copy(contour)
    cnt_new[:,:,0]=np.int32(cnt_new[:,:,0]-x_min+eps)
    cnt_new[:,:,1]=np.int32(cnt_new[:,:,1]-y_min+eps)

    x_max,y_max=np.max(cnt_new,axis=0)[0]

    mask = np.zeros((x_max+eps,y_max+eps))
    cv2.drawContours(mask, [cnt_new], -1, 255, thickness=cv2.FILLED)

    analyzer = StereoFractAnalyzer()

    sizes, counts = analyzer.box_count_image(Image.fromarray(np.uint8(cm.gist_earth(mask)*255)))
    log_sizes = np.log(1/sizes)
    log_counts = np.log(counts)
    coeffs = np.polyfit(log_sizes, log_counts, 1)
    fractal_dimension = coeffs[0]

    return fractal_dimension


binary, contours = preprocess_image(images_paths[0])

# Prepare data
new_contours=[contour for contour in contours if len(contour)>2]

indices = list(range(len(new_contours)))

# Process in parallel - NO shared_objects
fds = []
with WorkerPool(n_jobs=10) as pool:
    results = pool.map(process_contour, indices, progress_bar=True)
    fds = [fd for fd in results if fd is not None]

In [ ]:
plt.hist(fds,bins=50)
# plt.xlim(0,3)
plt.xlabel('fractal dimention')
plt.ylabel('count')
plt.title('Histogram of fractal dimension')
# plt.legend([f'{len(contours)} contours\n{images_path.split("/")[-1]}'],loc='upper right')
# plt.savefig(f'fractal_dimension_{images_path.split("/")[-1]}') 

In [ ]:

for step in [0.01,0.05,0.1,0.2]:

    plt.figure(figsize=(10,7))

    for image_path in tqdm(images_paths):
        binary, contours = preprocess_image(image_path)

        # Prepare data
        new_contours=[contour for contour in contours if len(contour)>2]

        indices = list(range(len(new_contours)))

        # Process in parallel - NO shared_objects
        fds = []
        with WorkerPool(n_jobs=n_jobs) as pool:
            results = pool.map(process_contour, indices, progress_bar=True)
            fds = [fd for fd in results if fd is not None]

        x, y = cstats.stats_preprocess(fds, step)
        plt.plot(x, y, '-o')

        legend.append(f'{len(contours)} contours {image_path.split("/")[-1]}')


    # plt.legend(legend, loc='upper right',fontsize=15)
    # plt.xlim(0,40)
    # plt.xscale('log')
    # plt.yscale('log')
    plt.xlabel('fractal dimension',fontsize=15)
    plt.ylabel('p(x)',fontsize=15)
    plt.title(f'step={step}',fontsize=15)

    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    # plt.savefig(f'fractal_dimension={step}.jpg',bbox_inches='tight')

    plt.show()

In [ ]:
%%time

def process_contour(contour_idx):
    ## global var!!!!!

    contour = contours[contour_idx]

    eps=3
    x_min,y_min=np.min(contour,axis=0)[0]

    cnt_new = copy.copy(contour)
    cnt_new[:,:,0]=np.int32(cnt_new[:,:,0]-x_min+eps)
    cnt_new[:,:,1]=np.int32(cnt_new[:,:,1]-y_min+eps)

    x_max,y_max=np.max(cnt_new,axis=0)[0]

    mask = np.zeros((x_max+eps,y_max+eps))
    cv2.drawContours(mask, [cnt_new], -1, 255, thickness=cv2.FILLED)

    analyzer = StereoFractAnalyzer()

    sizes, counts = analyzer.box_count_image(Image.fromarray(np.uint8(cm.gist_earth(mask)*255)))
    log_sizes = np.log(1/sizes)
    log_counts = np.log(counts)
    coeffs = np.polyfit(log_sizes, log_counts, 1)
    fractal_dimension = coeffs[0]

    return fractal_dimension


types=['fractal_dimension']
type=types[0]
n_jobs=20

# step=1

for step in tqdm([0.01,0.05,0.1,0.2]):

    legend=[]

    fig, axes= plt.subplots(2,4,figsize=(40,14))
    
    # Define 10 distinct colors
    colors = plt.cm.tab10(np.linspace(0, 1, 10))
    
    n_list=[]
    p_list=[]
    lambda_list=[]
    mu_list=[]
    sigma_list=[]
    a_list=[]

    for idx, image_path in enumerate(images_paths):

        binary, contours = preprocess_image(image_path)

        # Prepare data
        new_contours=[contour for contour in contours if len(contour)>2]

        indices = list(range(len(new_contours)))

        # Process in parallel - NO shared_objects
        len_list = []
        with WorkerPool(n_jobs=n_jobs) as pool:
            results = pool.map(process_contour, indices, progress_bar=True)
            len_list = [fd for fd in results if fd is not None]



        x_orig, y_orig = cstats.stats_preprocess(len_list, step)
        
        # Calculate mode (x value with maximum y)
        mode_idx = np.argmax(y_orig)
        mode = x_orig[mode_idx]
        
        # Get color for this image (cycle through colors if more than 10 images)
        color = colors[idx % len(colors)]

        # Fit binomial distribution
        (x_fit, y_fit), n, p, amp = capprox.binomial_approx(x_orig, y_orig, n=25,p=0.2, x_lim=[0,5], N=100)
        axes[0,0].plot(x_fit, y_fit, ':', linewidth=2, color=color)

        # Fit Poisson distribution
        (x_fit, y_fit), lam, amp_poisson = capprox.poisson_approx(x_orig, y_orig, x_lim=[-5,5], N=100)
        axes[0,1].plot(x_fit, y_fit, ':', linewidth=2, color=color)

        # Fit Gaussian distribution
        (x_fit, y_fit), mu, sigma, amp_gauss = capprox.gauss_approx(x_orig, y_orig, mu=3, sigma=3, amp=1, x_lim=[0,5], N=100)
        axes[0,2].plot(x_fit, y_fit, '-.', linewidth=2, color=color)

        # Fit Exponential distribution
        (x_fit, y_fit), a, amp_exp = capprox.exponential_approx(x_orig, y_orig, a=5, amp=1, x_lim=[0,5], N=100)
        axes[0,3].plot(x_fit, y_fit, '--', linewidth=2, color=color)

        axes[0, 0].set_title(f'binomial fit, step={step}',fontsize=15)
        axes[0,1].set_title(f'poisson fit, step={step}',fontsize=15)
        axes[0,2].set_title(f'gaussian fit, step={step}',fontsize=15)
        axes[0,3].set_title(f'exponential fit, step={step}',fontsize=15)
        
        n_list.append(n)
        p_list.append(p)
        lambda_list.append(lam)
        mu_list.append(mu)
        sigma_list.append(sigma)
        a_list.append(a)

        # Plot original data with the same color (include mode in legend)
        axes[0,0].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, n={n:.0f}, p={p:.3f}')
        axes[0,1].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, λ={lam:.2f}' )
        axes[0,2].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, μ={mu:.2f}, σ={sigma:.2f}' )
        axes[0,3].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, a={a:.2f}' )
    
    
    axes[1,0].set_title(f'binomial fit, step={step}',fontsize=15)
    axes[1,1].set_title(f'poisson fit, step={step}',fontsize=15)
    axes[1,2].set_title(f'gaussian fit, step={step}',fontsize=15)
    axes[1,3].set_title(f'exponential fit, step={step}',fontsize=15)

    # Use dual y-axis for n and p (n on left, p on right)
    ax1 = axes[1,0]  # Left axis for n
    ax1.plot(range(len(images_paths)), n_list, '-o', label='n', color='blue')
    ax1.set_ylabel('n', color='blue', fontsize=15)
    ax1.tick_params(axis='y', labelcolor='blue', labelsize=15)
    # axes[1,0].set_yscale('log')
    
    # Create second y-axis on the right for p
    ax2 = ax1.twinx()  # Right axis for p
    ax2.plot(range(len(images_paths)), p_list, '-o', label='p', color='red')
    ax2.set_ylabel('p', color='red', fontsize=15)
    ax2.tick_params(axis='y', labelcolor='red', labelsize=15)
    axes[1,0].twinx().set_yscale('log')
    
    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=15)
    
    axes[1,1].plot(range(len(images_paths)), lambda_list, '-o', label='λ')
    axes[1,2].plot(range(len(images_paths)), mu_list, '-o', label='mu')
    axes[1,2].plot(range(len(images_paths)), sigma_list, '-o', label='sigma')
    axes[1,3].plot(range(len(images_paths)), a_list, '-o', label='a')

    for ax in [axes[1,0], axes[1,1], axes[1,2], axes[1,3]]:
        ax.set_xticks(range(len(images_paths)))
        ax.set_xticklabels(images_names, rotation=45, fontsize=15, ha='right')

    axes[0,0].legend( loc='upper right',fontsize=15)
    axes[0,1].legend( loc='upper right',fontsize=15)
    axes[0,2].legend( loc='upper right',fontsize=15)
    axes[0,3].legend( loc='upper right',fontsize=15)
    axes[1,1].legend( loc='upper right',fontsize=15)
    axes[1,2].legend( loc='upper right',fontsize=15)
    axes[1,3].legend( loc='upper right',fontsize=15)

    # axes[0,0].set_ylim(0,0.2)
    # axes[0,1].set_ylim(0,0.2)
    # axes[0,2].set_ylim(0,0.2)
    # axes[0,3].set_ylim(0,0.2)

    axes[0,0].set_xlim(1,2)
    axes[0,1].set_xlim(1,2)
    axes[0,2].set_xlim(1,2)
    axes[0,3].set_xlim(1,2)

    if type=='len_nodes':
        axes[0,0].set_xlabel('len of contour, nodes',fontsize=15)
        axes[0,1].set_xlabel('len of contour, nodes',fontsize=15)
        axes[0,2].set_xlabel('len of contour, nodes',fontsize=15)
        axes[0,3].set_xlabel('len of contour, nodes',fontsize=15)
    elif type=='len_pixels':
        axes[0,0].set_xlabel('len of contour, pixels',fontsize=15)
        axes[0,1].set_xlabel('len of contour, pixels',fontsize=15)
        axes[0,2].set_xlabel('len of contour, pixels',fontsize=15)
        axes[0,3].set_xlabel('len of contour, pixels',fontsize=15)
    elif type=='area':
        axes[0,0].set_xlabel('area of contour, pixels',fontsize=15)
        axes[0,1].set_xlabel('area of contour, pixels',fontsize=15)
        axes[0,2].set_xlabel('area of contour, pixels',fontsize=15)
        axes[0,3].set_xlabel('area of contour, pixels',fontsize=15)

    axes[0,0].set_ylabel('p(x)',fontsize=15)
    axes[0,1].set_ylabel('p(x)',fontsize=15)
    axes[0,2].set_ylabel('p(x)',fontsize=15)
    axes[0,3].set_ylabel('p(x)',fontsize=15)

    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    if type=='len_nodes':
        plt.savefig(f'countour_lens_nodes_step={step}.jpg',bbox_inches='tight')
    elif type=='len_pixels':
        plt.savefig(f'countour_lens_pixels_step={step}.jpg',bbox_inches='tight')
    elif type=='area':
        plt.savefig(f'countour_areas_step={step}.jpg',bbox_inches='tight')

    plt.show()

In [ ]:
binary, cnts = preprocess_image(image_path)

len_list=[]

new_contours=[contour for contour in cnts if len(contour)>2]

In [ ]:
cnts